<a href="https://colab.research.google.com/github/yvesemmanuel/deep-learning/blob/main/vit-semantic-segmentation/vit-base-patch16-224-in21k-l0.005_b64_e5_finetuned-lora-food101-2025-11-20_02-34-57.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import logging
import sys

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

logger.handlers.clear()
logger.propagate = False

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

logger.addHandler(handler)

logger.info("Logger is working!")

2025-11-20 02:33:15,672 - INFO - Logger is working!


In [2]:
!pip install transformers accelerate evaluate datasets peft wandb fsspec==2023.9.2 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.4/173.4 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2023.9.2 which is incompatible.


# HuggingFace and Weights and Biases

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
import wandb
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: yveemmanuel (emmanuel-company) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [5]:
model_checkpoint = "google/vit-base-patch16-224-in21k"
dataset_id = "food101"

# Load dataset



> This dataset consists of 101 food categories, with 101'000 images. For each class, 250 manually reviewed test images are provided as well as 750 training images. [Read more](https://huggingface.co/datasets/ethz/food101).



In [6]:
from datasets import load_dataset

dataset = load_dataset(dataset_id)

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00008.parquet:   0%|          | 0.00/490M [00:00<?, ?B/s]

data/train-00001-of-00008.parquet:   0%|          | 0.00/464M [00:00<?, ?B/s]

data/train-00002-of-00008.parquet:   0%|          | 0.00/472M [00:00<?, ?B/s]

data/train-00003-of-00008.parquet:   0%|          | 0.00/464M [00:00<?, ?B/s]

...

data/validation-00000-of-00003.parquet:   0%|          | 0.00/423M [00:00<?, ?B/s]

data/validation-00001-of-00003.parquet:   0%|          | 0.00/413M [00:00<?, ?B/s]

data/validation-00002-of-00003.parquet:   0%|          | 0.00/426M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/75750 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/25250 [00:00<?, ? examples/s]

# Preprocessing dataset

This includes:

*   Labels mapping;
*   Data splitting;
*   Data normalization;
*   Data training augmentation.

## Label-to-ID mapping

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 75750
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 25250
    })
})

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 75750
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 25250
    })
})

In [9]:
labels = dataset["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
  label2id[label] = i
  id2label[i] = label

In [10]:
id2label[2]

'baklava'

## Image processor

In [11]:
from transformers import AutoImageProcessor

In [12]:
image_processor = AutoImageProcessor.from_pretrained(model_checkpoint)

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


## Training data augmentation & test/eval normalization

In [13]:
from torchvision.transforms import (
  CenterCrop,
  Compose,
  Normalize,
  RandomHorizontalFlip,
  RandomResizedCrop,
  Resize,
  ToTensor,
)

In [14]:
normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
train_transforms = Compose(
  [
      RandomResizedCrop(image_processor.size["height"]),
      RandomHorizontalFlip(),
      ToTensor(),
      normalize,
  ]
)

transforms = Compose(
  [
      Resize(image_processor.size["height"]),
      CenterCrop(image_processor.size["height"]),
      ToTensor(),
      normalize,
  ]
)


def preprocess_train(example_batch):
  """Apply train_transforms across a batch."""
  example_batch["pixel_values"] = [train_transforms(image.convert("RGB")) for image in example_batch["image"]]
  return example_batch


def preprocess(example_batch):
  """Apply transforms across a batch."""
  example_batch["pixel_values"] = [transforms(image.convert("RGB")) for image in example_batch["image"]]
  return example_batch

## Data splitting

Three data splits; 1) train, 2) eval, 2) test.

In [15]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 75750
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 25250
    })
})

In [16]:
train_data = dataset["train"]
val_data = dataset["validation"]

val_data_split = val_data.train_test_split(test_size=0.3)
val_data = val_data_split["train"]
test_data = val_data_split["test"]

In [17]:
train_data.set_transform(preprocess_train)
val_data.set_transform(preprocess)
test_data.set_transform(preprocess)

# Model *tunable* hyperparameters

In [18]:
def print_trainable_parameters(model):
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
      all_param += param.numel()
      if param.requires_grad:
          trainable_params += param.numel()
  logger.info(
      f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
  )

In [19]:
from transformers import AutoModelForImageClassification

In [20]:
model = AutoModelForImageClassification.from_pretrained(
  model_checkpoint,
  label2id=label2id,
  id2label=id2label,
  ignore_mismatched_sizes=True,  # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
)

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
print_trainable_parameters(model)

2025-11-20 02:34:56,543 - INFO - trainable params: 85876325 || all params: 85876325 || trainable%: 100.00


# Low-Rank Adaptation

In [22]:
from peft import LoraConfig, get_peft_model

In [23]:
lora_config = LoraConfig(
  r=32,
  lora_alpha=16,
  target_modules=["query", "value"],
  lora_dropout=0.1,
  bias="none",
  modules_to_save=["classifier"],
)
lora_model = get_peft_model(model, lora_config)

In [24]:
print_trainable_parameters(lora_model)

2025-11-20 02:34:56,959 - INFO - trainable params: 1257317 || all params: 87133642 || trainable%: 1.44


# Training

In [25]:
from transformers import TrainingArguments
import datetime

In [26]:
import datetime

In [27]:
model_name = model_checkpoint.split("/")[-1]
batch_size = 64
learning_rate = 5e-3
num_train_epochs = 5
logging_steps = 10

timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
run_name = f"{model_name}-l{learning_rate}_b{batch_size}_e{num_train_epochs}_finetuned-lora-{dataset_id}-{timestamp}"

training_args = TrainingArguments(
  remove_unused_columns=False,
  eval_strategy="epoch",
  save_strategy="epoch",
  learning_rate=learning_rate,
  per_device_train_batch_size=batch_size,
  gradient_accumulation_steps=4,
  per_device_eval_batch_size=batch_size,
  fp16=True,
  num_train_epochs=num_train_epochs,
  logging_steps=logging_steps,
  load_best_model_at_end=True,
  metric_for_best_model="accuracy",
  push_to_hub=True,
  label_names=["labels"],
  run_name=run_name,
  report_to=["wandb"]
)

## Evaluation metrics

In [28]:
import numpy as np
import evaluate

In [29]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
  """Computes accuracy on a batch of predictions"""
  predictions = np.argmax(eval_pred.predictions, axis=1)
  return metric.compute(predictions=predictions, references=eval_pred.label_ids)

## Data collaction function

> A collation function is used by Trainer to gather a batch of training and evaluation examples and prepare them in a format that is acceptable by the underlying model. [Read more](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer).


In [30]:
import torch

In [31]:
def collate_fn(examples):
  pixel_values = torch.stack([example["pixel_values"] for example in examples])
  labels = torch.tensor([example["label"] for example in examples])
  return {"pixel_values": pixel_values, "labels": labels}

## Training steps

In [32]:
from transformers import Trainer

In [33]:
trainer = Trainer(
  lora_model,
  training_args,
  train_dataset=train_data,
  eval_dataset=val_data,
  tokenizer=image_processor,
  compute_metrics=compute_metrics,
  data_collator=collate_fn,
)
train_results = trainer.train()

/tmp/ipython-input-347704202.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.864800,0.482838,0.863423
2,0.754200,0.420630,0.881810
3,0.654400,0.377245,0.889109
4,0.549200,0.349105,0.899463
5,0.484600,0.332042,0.904781


/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))
/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))
/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))
/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))
/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


# Evaluate metrics on test - unseen - dataset

Results from training, system usage and evaluation will be in the dashboard of Weights and Biases. [Read more](https://wandb.ai/emmanuel-company/huggingface/runs/ppf670zf?nw=nwuseryveemmanuel).

In [34]:
trainer.evaluate(test_data)

/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


{'eval_loss': 0.3286559283733368,
 'eval_accuracy': 0.905082508250825,
 'eval_runtime': 46.0502,
 'eval_samples_per_second': 164.494,
 'eval_steps_per_second': 2.584,
 'epoch': 5.0}

In [35]:
wandb.finish()

eval/accuracy,▁▄▅▇██
eval/loss,█▅▃▂▁▁
eval/runtime,█████▁
eval/samples_per_second,█▁▇▇▂▃
eval/steps_per_second,█▁▇▇▂▄
train/epoch,▁▁▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█
train/global_step,▁▁▁▁▁▂▂▂▂▂▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train/grad_norm,▁▂▁▂▂▂▂█▂▂▂▂▂▂▄▂▂▂▁▂▂▁▂▁▁▁▂▁▁▂▂▁▁▂▁▂▂▂▂▂
train/learning_rate,█████▇▇▇▆▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁
train/loss,█▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/accuracy,0.90508
